## Chassisml Example Notebooks
Welcome to the examples section for [Chassis.ml](https://chassis.ml), which contains notebooks that leverage Chassisml to auto-containerize models built using the most common machine learning frameworks. 

**NOTE:** Chassisml provides two key functionalities: 
1. Create a Docker container from your model code and push that container image to a Docker registry. This is the default behavior.
2. Should you pass valid Modzy credentials as optional parameters, Chassisml will take the container and upload it directly to the Modzy environment you specify. You will notice most of these notebooks deploy the model to one of the Modzy internal development environments.   

Can't find the framework you are looking for or need help? Fork this repository and open a PR, we're always interested in growing this example bank! 

The primary maintainers of Chassis also actively monitor our [Discord Server](https://discord.gg/tdfXFY2y), so feel free to join and ask any questions you might have. We'll be there to respond and help out promptly. 

In [1]:
import chassisml
import numpy as np
import getpass
import json
from io import StringIO
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn_pmml_model.ensemble import PMMLForestClassifier

## Enter credentials
Dockerhub creds and Modzy API Key

In [2]:
dockerhub_user = getpass.getpass('docker hub username')
dockerhub_pass = getpass.getpass('docker hub password')
modzy_api_key = getpass.getpass('modzy api key')

docker hub username········
docker hub password········
modzy api key········


## Load and Test Model from PMML File

In [3]:
# Prepare data
iris = load_iris()
X = pd.DataFrame(iris.data)
X.columns = np.array(iris.feature_names)
y = pd.Series(np.array(iris.target_names)[iris.target])
y.name = "Class"
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.33, random_state=123)

# Create sample data for testing later
with open("data/sample_iris.csv", "w") as f:
    Xte[:10].to_csv(f, index=False)

In [4]:
# Load model
clf = PMMLForestClassifier(pmml="models/randomForest.pmml")
labels = clf.classes_.tolist()

# Test model
clf.predict(Xte)
clf.score(Xte, yte)

1.0

## Prepare context dict
Initialize anything here that should persist across inference runs

In [8]:
def preprocess_inputs(raw_input_bytes):
    # load data
    inputs = pd.read_csv(StringIO(str(raw_input_bytes, "utf-8")))
    return inputs

def postprocess_outputs(raw_predictions):
    # process output
    inference_result = {
        "result":[
            {
                "row": i+1,
                "classPredictions": [
                    {"class": context['labels'][idx], "score": results[idx]}
                    for idx in np.argsort(results)[::-1]
                ]  
            } for i, results in enumerate(raw_predictions)
        ] 

    }    
    
    
    # format output
    structured_output = {
        "data": {
            "result": inference_result["result"],
            "explanation": None,
            "drift": None,
        }
    }
    
    return structured_output

In [9]:
# This will be passed to Chassis:
context = {
    "model": clf,
    "labels": labels,
    "preprocess": preprocess_inputs,
    "postprocess": postprocess_outputs
}

## Write process function

* Must take bytes and context dict as input
* Preprocess bytes, run inference, postprocess model output, return results

In [10]:
def process(input_bytes,context):
    # load data
    inputs = context["preprocess"](input_bytes)
    # make predictions
    output = context["model"].predict_proba(inputs)
    # process output
    structured_output = context["postprocess"](output)
    return structured_output

## Initialize Chassis Client
We'll use this to interact with the Chassis service

In [11]:
chassis_client = chassisml.ChassisClient("http://localhost:5000")

## Create and test Chassis model
* Requires `context` dict containing all variables which should be loaded once and persist across inferences
* Requires `process_fn` defined above

In [12]:
# create Chassis model
chassis_model = chassis_client.create_model(context=context,process_fn=process)

# test Chassis model locally (can pass filepath, bufferedreader, bytes, or text here):
sample_filepath = './data/sample_iris.csv'
results = chassis_model.test(sample_filepath)
print(results)

b'{"data":{"result":[{"row":1,"classPredictions":[{"class":"versicolor","score":0.76},{"class":"virginica","score":0.24},{"class":"setosa","score":0.0}]},{"row":2,"classPredictions":[{"class":"virginica","score":0.995},{"class":"versicolor","score":0.005},{"class":"setosa","score":0.0}]},{"row":3,"classPredictions":[{"class":"virginica","score":1.0},{"class":"versicolor","score":0.0},{"class":"setosa","score":0.0}]},{"row":4,"classPredictions":[{"class":"versicolor","score":1.0},{"class":"virginica","score":0.0},{"class":"setosa","score":0.0}]},{"row":5,"classPredictions":[{"class":"setosa","score":1.0},{"class":"virginica","score":0.0},{"class":"versicolor","score":0.0}]},{"row":6,"classPredictions":[{"class":"virginica","score":0.82},{"class":"versicolor","score":0.18},{"class":"setosa","score":0.0}]},{"row":7,"classPredictions":[{"class":"versicolor","score":0.995},{"class":"virginica","score":0.005},{"class":"setosa","score":0.0}]},{"row":8,"classPredictions":[{"class":"setosa","sc

In [13]:
# test environment and model within Chassis service, must pass filepath here:

# dry run before build
test_env_result = chassis_model.test_env(sample_filepath)
print(test_env_result)

Starting test job... Ok!
{'model_output': 'Single input prediction:\n\nb\'{"data":{"result":[{"row":1,"classPredictions":[{"class":"versicolor","score":0.76},{"class":"virginica","score":0.24},{"class":"setosa","score":0.0}]},{"row":2,"classPredictions":[{"class":"virginica","score":0.995},{"class":"versicolor","score":0.005},{"class":"setosa","score":0.0}]},{"row":3,"classPredictions":[{"class":"virginica","score":1.0},{"class":"versicolor","score":0.0},{"class":"setosa","score":0.0}]},{"row":4,"classPredictions":[{"class":"versicolor","score":1.0},{"class":"virginica","score":0.0},{"class":"setosa","score":0.0}]},{"row":5,"classPredictions":[{"class":"setosa","score":1.0},{"class":"virginica","score":0.0},{"class":"versicolor","score":0.0}]},{"row":6,"classPredictions":[{"class":"virginica","score":0.82},{"class":"versicolor","score":0.18},{"class":"setosa","score":0.0}]},{"row":7,"classPredictions":[{"class":"versicolor","score":0.995},{"class":"virginica","score":0.005},{"class":"s

## Publish model to Modzy
Need to provide model name, model version, Dockerhub credentials, and required Modzy info

In [ ]:
MODZY_URL = "https://demo.modzy.engineering/api"

response = chassis_model.publish(
    model_name="PMML Random Forest Iris Classification",
    model_version="0.0.1",
    registry_user=dockerhub_user,
    registry_pass=dockerhub_pass,
    modzy_sample_input_path=sample_filepath,
    modzy_api_key=modzy_api_key,
    modzy_url=MODZY_URL
)

In [37]:
job_id = response.get('job_id')
final_status = chassis_client.block_until_complete(job_id)

In [42]:
if final_status["result"] is not None:
    print("New model URL: {}".format(final_status["result"]["container_url"]))
else:
    print("Chassis job failed \n\n {}".format(final_status))

New model URL: https://demo.modzy.engineering/models/ocljmj1dvy/0.0.1


## Run sample job using Modzy SDK
Submit inference job to our newly-deploy model running on Modzy

In [ ]:
from modzy import ApiClient

client = ApiClient(base_url=MODZY_URL, api_key=modzy_api_key)

input_name = final_status['result']['inputs'][0]['name']
model_id = final_status['result'].get("model").get("modelId")
model_version = final_status['result'].get("version")

inference_job = client.jobs.submit_file(model_id, model_version, {input_name: sample_filepath})
inference_job_result = client.results.block_until_complete(inference_job, timeout=None)
inference_job_results_json = inference_job_result.get_first_outputs()['results.json']
print(inference_job_results_json)